In [ ]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings("ignore")

import glob
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import tensorflow as tf
import keras
from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import model_from_json


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score

import optuna

import psycopg2
from config import *

import utm
import time

from sklearn.ensemble import RandomForestClassifier
import optuna

In [ ]:
def get_closest(lon,lat,estaciones):
    distancias_list=[]
    cercano=0
    closest=10000000
    for i in range(len(estaciones)):
        dist=(estaciones['longitude'].iloc[i]-lon)**2+(estaciones['latitude'].iloc[i]-lat)**2
        distancias_list.append(dist)
        if(dist<closest):
            closest=dist
            cercano=estaciones['nombrecorto'].iloc[i]
    estaciones['distancias']=distancias_list
    return cercano

In [ ]:
conexion = psycopg2.connect(database=db_database, 
                                user=db_user, 
                                password=db_password, 
                                host=db_host, 
                                port=db_port)

In [ ]:
query='''select date, variedad, min(phenologystageid) as phenologystageid, codigocatastro as codigo 
from redfara.redfara_fenologia
where especie='VIÑEDO VINIFICACION'
group by date, variedad, codigo, phenologystageid;'''
phenological_data = pd.read_sql_query(query, con=conexion).drop_duplicates()
conexion.commit()
phenological_data

In [ ]:
query='select * from cadastral.parcelas where codigo in '+ str(list(phenological_data.codigo.unique())).replace('[','(').replace(']',')') + ';'
cadastral_data = pd.read_sql_query(query, con=conexion)
conexion.commit()


In [ ]:
cadastral_data['longitude']=cadastral_data['coordenadas_epsgwgs84'].apply(lambda x:float(x[0]))
cadastral_data['latitude']=cadastral_data['coordenadas_epsgwgs84'].apply(lambda x:float(x[1]))
cadastral_data=cadastral_data[['codigo', 'longitude','latitude','altitud']].drop_duplicates()
cadastral_data

In [ ]:
query='select * from public.estara;'
stations = pd.read_sql_query(query, con=conexion).drop_duplicates()
conexion.commit()
stations

In [ ]:
stations['longitude']=stations.longitud.str.split("'|º").apply(lambda x:int(x[0])+int(x[1])/60+int(x[2])/3600000)
stations['longitude']=stations['longitude']*(stations.longitud.str.contains('E')*2-1)
stations['latitude']=stations.latitud.str.split("'|º").apply(lambda x:int(x[0])+int(x[1])/60+int(x[2])/3600000)
stations['latitude']=stations['latitude']*(stations.latitud.str.contains('N')*2-1)
stations=stations[['nombrecorto','longitude','latitude', 'altitud']]
stations

In [ ]:
cercanias=[]
for i in range(len(cadastral_data)):
    cercanias.append(get_closest(cadastral_data.iloc[i].longitude,cadastral_data.iloc[i].latitude,stations))
cadastral_data['closest']=cercanias
cadastral_data

In [ ]:
closest_stations = stations[stations.nombrecorto.isin(cadastral_data.closest.unique())]
closest_stations

In [ ]:
lowest_year=phenological_data.date.dt.year.min()
lowest_year

In [ ]:
query='select * from public.meteorological_data WHERE anio >= ' + str(lowest_year) + ' AND estacion in ' + str(list(closest_stations.nombrecorto.unique())).replace('[','(').replace(']',')') + ';'
meteorological_data = pd.read_sql_query(query, con=conexion).drop_duplicates()
conexion.commit()
meteorological_data = meteorological_data.drop(['ubi', 'season', 'hourFrac_sum'], axis=1)
variables_diarias_min=['tmed_min', 'rad_min']
variables_diarias_max=['tmed_max', 'rad_max']
variables_diarias_mean=['tmed_mean', 'rad_mean', 'wind_N', 'wind_NE', 'wind_E','wind_SE', 'wind_S', 'wind_SW', 
                        'wind_W', 'wind_NW']
variables_semanales=['gdd_4.5_t0_Tbase_sum',
       'gdd_4.5_t0_TbaseMax_sum', 'gdd_4.5_1_Tbase_sum',
       'gdd_4.5_1_TbaseMax_sum', 'gdd_4.5_2_Tbase_sum',
       'gdd_4.5_2_TbaseMax_sum', 'gdd_10.0_t0_Tbase_sum',
       'gdd_10.0_t0_TbaseMax_sum', 'gdd_10.0_1_Tbase_sum',
       'gdd_10.0_1_TbaseMax_sum', 'gdd_10.0_2_Tbase_sum',
       'gdd_10.0_2_TbaseMax_sum', 'chillingDD_7.0_t0_Tbase_sum',
       'chillingDD_7.0_t0_Tbasemin_sum', 'chillingDD_7.0_t0_Utah_sum',
       'chillingDD_7.0_1_Tbase_sum', 'chillingDD_7.0_1_Tbasemin_sum',
       'chillingDD_7.0_1_Utah_sum', 'chillingDD_7.0_2_Tbase_sum',
       'chillingDD_7.0_2_Tbasemin_sum', 'chillingDD_7.0_2_Utah_sum', 'rad_sum',
       'precip_sum', 'winkler_4.5_Tbase', 'winkler_4.5_TbaseMax',
       'winkler_10.0_Tbase', 'winkler_10.0_TbaseMax',
       'gdd_4.5_t0_Tbase_sum_Cumm', 'gdd_4.5_t0_TbaseMax_sum_Cumm',
       'gdd_4.5_1_Tbase_sum_Cumm', 'gdd_4.5_1_TbaseMax_sum_Cumm',
       'gdd_4.5_2_Tbase_sum_Cumm', 'gdd_4.5_2_TbaseMax_sum_Cumm',
       'gdd_10.0_t0_Tbase_sum_Cumm', 'gdd_10.0_t0_TbaseMax_sum_Cumm',
       'gdd_10.0_1_Tbase_sum_Cumm', 'gdd_10.0_1_TbaseMax_sum_Cumm',
       'gdd_10.0_2_Tbase_sum_Cumm', 'gdd_10.0_2_TbaseMax_sum_Cumm',
       'chillingDD_7.0_t0_Tbase_sum_Cumm',
       'chillingDD_7.0_t0_Tbasemin_sum_Cumm',
       'chillingDD_7.0_t0_Utah_sum_Cumm', 'chillingDD_7.0_1_Tbase_sum_Cumm',
       'chillingDD_7.0_1_Tbasemin_sum_Cumm', 'chillingDD_7.0_1_Utah_sum_Cumm',
       'chillingDD_7.0_2_Tbase_sum_Cumm', 'chillingDD_7.0_2_Tbasemin_sum_Cumm',
       'chillingDD_7.0_2_Utah_sum_Cumm', 'rad__t0__Cumm', 'rad__1__Cumm',
       'rad__2__Cumm', 'precip__t0__Cumm', 'precip__1__Cumm',
       'precip__2__Cumm', 'winkler_4.5_t0_Tbase_Cumm',
       'winkler_4.5_t0_TbaseMax_Cumm', 'winkler_4.5_1_Tbase_Cumm',
       'winkler_4.5_1_TbaseMax_Cumm', 'winkler_4.5_2_Tbase_Cumm',
       'winkler_4.5_2_TbaseMax_Cumm', 'winkler_10.0_t0_Tbase_Cumm',
       'winkler_10.0_t0_TbaseMax_Cumm', 'winkler_10.0_1_Tbase_Cumm',
       'winkler_10.0_1_TbaseMax_Cumm', 'winkler_10.0_2_Tbase_Cumm',
       'winkler_10.0_2_TbaseMax_Cumm']
meteorological_data

In [ ]:
meteorological_data.columns

In [ ]:
datos_meteo_buenos_list=[]
n_dias_atras=15
n_dias_alante=8
for estacion in meteorological_data['estacion'].unique():
    datos_est=meteorological_data[meteorological_data.estacion==estacion].sort_values('fecha').set_index('fecha')
    datos_meteo_buenos_est_list=[datos_est[['estacion']]]
    for var in variables_diarias_min:
        datos_var_est=datos_est[[var]].resample('1D').min()
        for i in range(1,n_dias_atras):
            datos_var_est[var + " " + str(i) + "_dias_atras"]=datos_var_est[var].resample('1D').min().shift(i)
        for i in range(1,n_dias_alante):
            datos_var_est[var + " " + str(i) + "_dias_adelante"]=datos_var_est[var].resample('1D').min().shift(-i)
        datos_meteo_buenos_est_list.append(datos_var_est)
    for var in variables_diarias_max:
        datos_var_est=datos_est[[var]].resample('1D').max()
        for i in range(1,n_dias_atras):
            datos_var_est[var + " " + str(i) + "_dias_atras"]=datos_var_est[var].resample('1D').max().shift(i)
        for i in range(1,n_dias_alante):
            datos_var_est[var + " " + str(i) + "_dias_adelante"]=datos_var_est[var].resample('1D').max().shift(-i)
        datos_meteo_buenos_est_list.append(datos_var_est)
    for var in variables_diarias_mean:
        datos_var_est=datos_est[[var]].resample('1D').max()
        for i in range(1,n_dias_atras):
            datos_var_est[var + " " + str(i) + "_dias_atras"]=datos_var_est[var].resample('1D').max().shift(i)
        for i in range(1,n_dias_alante):
            datos_var_est[var + " " + str(i) + "_dias_adelante"]=datos_var_est[var].resample('1D').max().shift(-i)
        datos_meteo_buenos_est_list.append(datos_var_est)
    for var in variables_semanales:
        datos_var_est=datos_est[[var]].resample('1D').max()
        for i in range(1,1+n_dias_atras//7):
            datos_var_est[var + " " + str(i) + "_semanas_atras"]=datos_var_est[var].resample('1D').ffill().shift(i*7)
        for i in range(1,1+n_dias_alante//7):
            datos_var_est[var + " " + str(i) + "_semanas_adelante"]=datos_var_est[var].resample('1D').ffill().shift(-i*7)
        datos_meteo_buenos_est_list.append(datos_var_est)
        
    datos_var_est=pd.concat(datos_meteo_buenos_est_list,axis=1).reset_index()
    datos_meteo_buenos_list.append(datos_var_est)
datos_meteo_buenos=pd.concat(datos_meteo_buenos_list).dropna()
datos_meteo_buenos

In [ ]:
datos_part=pd.merge(phenological_data.reset_index(), cadastral_data, left_on='codigo', right_on='codigo', how='outer', indicator=True)
datos_part

In [ ]:
datos_part2=datos_part[datos_part._merge=='both']
datos_part2

In [ ]:
datos_campos=[]
for campo in datos_part2.codigo.unique():
    datos_part3=datos_part2[datos_part2.codigo==campo]
    datos_campos.append(datos_part3.sort_values('date').set_index('date').resample('1D').ffill().dropna())
    
datos_part=pd.concat(datos_campos).reset_index()
datos_part

In [ ]:
datos_part['anio']=datos_part['date'].dt.year
datos_part['dia']=datos_part['date'].dt.dayofyear
datos_part=datos_part.drop(columns=['index','_merge','date'])
datos_part=pd.get_dummies(datos_part,columns=['variedad'])

In [ ]:
datos_meteo_buenos['anio']=datos_meteo_buenos.fecha.dt.year
datos_meteo_buenos['dia']=datos_meteo_buenos.fecha.dt.dayofyear
datos_total=pd.merge(datos_part, datos_meteo_buenos, left_on=['closest','anio','dia'], right_on=['estacion','anio','dia'])
datos_total=datos_total.drop(['closest','fecha','estacion'],axis=1)
datos_total

In [ ]:
query='''select codigo, date, AVG(min) as min, AVG(max) as max, AVG(mean) as mean, AVG(std) as std, AVG(meidan) as median from 
public.copernicus_nvdi where pixels_array is not null and tesela is not null
group by codigo, date;'''
satelital_data = pd.read_sql_query(query, con=conexion).drop_duplicates()
conexion.commit()
satelital_data

In [ ]:
subdatas_list=[]
for campo in satelital_data.codigo.unique():
    subdata=satelital_data[satelital_data.codigo==campo]
    subdata['date']=pd.to_datetime(subdata['date'],format='%Y-%m-%d')
    subdata['date2']=subdata['date']
    subdata=subdata.sort_values('date').set_index('date').resample('1D').ffill().reset_index()
    subdata['diff']=(subdata['date']-subdata['date2']).dt.days
    subdatas_list.append(subdata[subdata['diff']<21].drop(columns=['diff','date2']))
satelital_data2=pd.concat(subdatas_list)
satelital_data2['dia']=satelital_data2.date.dt.dayofyear
satelital_data2['anio']=satelital_data2.date.dt.year
satelital_data2=satelital_data2.drop(columns=['date'])
satelital_data2

In [ ]:
conexion.close()

In [ ]:
datos_total2=pd.merge(datos_total, satelital_data2, left_on=['codigo','anio','dia'], right_on=['codigo','anio','dia'])
# datos_total2=datos_total2.drop(columns=['codigo'])
datos_total2

In [ ]:
datos_total2.columns[400:500]

In [ ]:
datos_total2.to_csv('/data/proyectos/GRAPEVINE/all_data.csv')

In [ ]:
def eval_model(model,model_chars,data):
    variedades=[col for col in data.columns if 'variedad_' in col]
    variables_basic=['phenologystageid','dia', 'min', 'max', 'mean', 'std', 'median']+variedades
    medidas=variables_basic
    
    altitud=model_chars['params_Altitud']
    latitud=model_chars['params_Latitud']
    longitud=model_chars['params_Longitud']
    if(longitud):
        medidas.append('longitude')
    if(latitud):
        medidas.append('latitude')
    if(altitud):
        medidas.append('altitud')
        
    if model_chars['params_Acumuladas']:
        
        inicio=model_chars['params_Inicio mediciones']
        suffix=''
        name_vars=[]
        
        chilling=model_chars['params_Chilling']
        winkler=model_chars['params_Winkler']
        gdd=model_chars['params_gdd']
        
        acumulativo=model_chars['params_Acumulativo']
        if (acumulativo):
            suffix='_Cumm'
            if model_chars['params_Precipitaciones']:
                name_vars.append('precip__'+inicio+'_')
            if model_chars['params_Radiacion']:
                name_vars.append('rad__'+inicio+'_')
                
        if (winkler|gdd):
            tbase=model_chars['params_Tbase']
            temperatura_inicio=str(model_chars['params_Temperatura inicio'])
        if (chilling):
            tbase_chill=model_chars['params_Tbase_chilling']
            name_vars.append('chillingDD_7.0_'+inicio+'_' + tbase_chill +'_sum')
        if (winkler):
            if acumulativo:
                name_vars.append('winkler_'+temperatura_inicio+'_'+inicio+'_' + tbase)
            else:
                name_vars.append('winkler_'+temperatura_inicio+'_' + tbase)
        if (gdd):
            name_vars.append('gdd_'+temperatura_inicio+'_'+inicio+'_' + tbase+'_sum')
       
        if len(name_vars)>0:
            semanas_list=[]
            for i in range(1,3):
                if model_chars['params_'+str(i)+'_semanas_atras']:
                    semanas_list.append(' '+str(i)+'_semanas_atras')
            if model_chars['params_1_semanas_adelante']:
                    semanas_list.append(' 1_semanas_adelante')
                    
            for name_var in name_vars:
                medidas.append(name_var+suffix)
                for semana in semanas_list:
                    medidas.append(name_var+suffix+semana)
        
    else:
        name_vars=[]    
        if model_chars['params_Temperatura']:
            if model_chars['params_Temperatura_media']:
                name_vars.append('tmed_mean')
            if model_chars['params_Temperatura_min']:
                name_vars.append('tmed_min')
            if model_chars['params_Temperatura_max']:
                name_vars.append('tmed_max')

        if model_chars['params_Radiacion']:
            if model_chars['params_Radiacion_media']:
                name_vars.append('rad_mean')
            if model_chars['params_Radiacion_min']:
                name_vars.append('rad_min')
            if model_chars['params_Radiacion_max']:
                name_vars.append('rad_max')

        if model_chars['params_Viento']:
            if model_chars['params_Viento_norte']:
                name_vars.append('wind_N')
            if model_chars['params_Viento_noreste']:
                name_vars.append('wind_NE')
            if model_chars['params_Viento_este']:
                name_vars.append('wind_E')
            if model_chars['params_Viento_sureste']:
                name_vars.append('wind_SE')
            if model_chars['params_Viento_sur']:
                name_vars.append('wind_S')
            if model_chars['params_Viento_suroeste']:
                name_vars.append('wind_SW')
            if model_chars['params_Viento_oeste']:
                name_vars.append('wind_W')
            if model_chars['params_Viento_noroeste']:
                name_vars.append('wind_NW')
        
        if len(name_vars)>0:
            dias_vars=[]
            for i in range(1,15):
                if model_chars['params_'+str(i)+'_dias_atras']:
                    dias_vars.append(' ' + str(i) + '_dias_atras')
            for i in range(1,8):
                if model_chars['params_'+str(i)+'_dias_adelante']:
                    dias_vars.append(' ' + str(i) + '_dias_adelante')

            for name_var in name_vars:
                medidas.append(name_var)
                for dia_var in dias_vars:
                    medidas.append(name_var+dia_var)

    medidas.append('dias_hasta')
    data=data[medidas]#.dropna()
                   
    
    X_vali=data.drop(['dias_hasta'], axis=1).values
    Y_vali=data['dias_hasta'].values

    
    
    preds = model.predict(X_vali)

    SCORE=r2_score(Y_vali,preds)
    print("R^2: ",SCORE)
    
    return SCORE, medidas, preds

In [ ]:
datos_no_val=datos_total2[datos_total2.anio<=2020]
datos_no_val.phenologystageid=datos_no_val.phenologystageid.replace({9.0:0.0})
datos_val=datos_total2[datos_total2.anio>2020]
datos_val.phenologystageid=datos_val.phenologystageid.replace({9.0:0.0})

In [ ]:
mejores_modelos={}

for estado_fen in range(1,9):
    temp=datos_val.copy()

    datos_list=[]
    for campaña in temp.anio.unique():
        datos_camp=temp[temp['anio']==campaña]
        for id_terr in datos_camp.codigo.unique():
            datos_camp_terr=datos_camp[datos_camp['codigo']==id_terr]
            x=datos_camp_terr[datos_camp_terr['phenologystageid']>=estado_fen]['dia'].values
            momento=0
            if len(x>0):
                momento=np.min(x)
            datos_camp_terr['dias_hasta']=momento-datos_camp_terr['dia']
            datos_list.append(datos_camp_terr[datos_camp_terr['dias_hasta']>0])

    datos_check=pd.concat(datos_list)
#     datos_check=datos_check[datos_check.anio==2018]

    modelos_estado=pd.read_csv(f'/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + 
                               str(estado_fen) + '/resumen_optuna-estado'+str(estado_fen)+'-r2-dias.csv')
    modelos_estado=modelos_estado.sort_values('value',ascending=False)
    modelo_elegido=modelos_estado.iloc[0]
    
    
    if (len(glob.glob('/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen) + '/model-estado'+ str(estado_fen) +'-'+str(modelo_elegido['value'])[:10]+'*.json'))>0):
        mejores_modelos[estado_fen]={}
        mejores_modelos[estado_fen]['dias_hasta']=datos_check['dias_hasta']
        mejores_modelos[estado_fen]['params']=modelo_elegido
        json_file = open(glob.glob('/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen) + '/model-estado'+ str(estado_fen) +'-'+str(modelo_elegido['value'])[:10]+'*.json')[0], 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        # load weights into new model
        model.load_weights(glob.glob('/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen)  + '/model-estado'+ str(estado_fen) +'-'+str(modelo_elegido['value'])[:10]+'*.h5')[0])
        mejores_modelos[estado_fen]['modelo']=model

        print(modelo_elegido['value'])
        mejores_modelos[estado_fen]['value'], mejores_modelos[estado_fen]['medidas'], mejores_modelos[estado_fen]['preds']=eval_model(model,modelo_elegido,datos_check)
    else:
        print("No hay modelos para el estado fenológico:", estado_fen)
    print('#'*10)

In [ ]:
import matplotlib.pyplot as plt
with pd.ExcelWriter("predicciones_y reales_por_estados_para_francis.xlsx") as writer:
    for estado_fen in range(1,9):
        plt.hist(mejores_modelos[estado_fen]['dias_hasta']-mejores_modelos[estado_fen]['preds'][:,0], bins=30)
        plt.title('Phenological State ' + str(estado_fen))
        plt.xlabel("Error")
        plt.ylabel("Count")
        plt.show()
        pd.DataFrame([mejores_modelos[estado_fen]['dias_hasta'].values,mejores_modelos[estado_fen]['preds'][:,0]],
                 index=['dias_hasta','prediction']).T.to_excel(writer, sheet_name='Estado_feno_'+str(estado_fen))

In [ ]:
for estado_fen in range(1,9):
    plt.hist(mejores_modelos[estado_fen]['dias_hasta']-np.mean(mejores_modelos[estado_fen]['dias_hasta']), bins=30)
    plt.title('Phenological State ' + str(estado_fen))
    plt.xlabel("Difference with the mean value")
    plt.ylabel("Count")
    plt.show()

In [ ]:
errors_by_day=[]
for estado_fen in range(1,9):
    df=pd.DataFrame([mejores_modelos[estado_fen]['dias_hasta'].values,mejores_modelos[estado_fen]['preds'][:,0]],
                 index=['days to phenological state','prediction']).T
    df['error_estado_feno'+str(estado_fen)]=abs(df['days to phenological state']-df['prediction'])
    df.groupby('days to phenological state').mean()[['error_estado_feno'+str(estado_fen)]].sort_index().plot(legend=False,
                                            title='Errors relative to phenological state ' + str(estado_fen) + ' proximity',
                                            ylabel='Mean error')
    errors_by_day.append(df.groupby('days to phenological state').mean()[['error_estado_feno'+str(estado_fen)]])
    
pd.concat(errors_by_day,axis=1).to_csv('errores_medios_por_dia_para_Francisco.csv')

In [ ]:
for estado_fen in range(1,9):
    print("VARIABLES UTILIZADAS POR EL MODELO QUE PREDICE EL ESTADO FENOLÓGICO", estado_fen, ":")
    print('-'*120)
    print(mejores_modelos[estado_fen]['medidas'])
    print('#'*120)

In [ ]:
pd.DataFrame(r2,index=range(1,9)).to_csv('r2_para_francisco.csv')

In [ ]:
ax = pd.DataFrame(r2,index=range(1,9)).plot(kind='bar', ylim=(0.7,1), legend=False, width=0.8, 
                                            title='R^2 for each phenology state')
for p in ax.patches:
    ax.annotate(str(np.round(p.get_height(),3)), (p.get_x() * 1.005, p.get_height() * 1.005))

In [ ]:
temp=datos_val.copy()

datos_list=[]
for campaña in temp.anio.unique():
    datos_camp=temp[temp['anio']==campaña]
    for id_terr in datos_camp.codigo.unique():
        datos_camp_terr=datos_camp[datos_camp['codigo']==id_terr].sort_values('dia')
        datos_camp_terr['future_pheno']=datos_camp_terr['phenologystageid'].shift(7)
        datos_list.append(datos_camp_terr.dropna())
datos_final=pd.concat(datos_list)

In [ ]:
datos_final=datos_final[datos_final.future_pheno>=datos_final.phenologystageid]
datos_final=datos_final[datos_final.future_pheno<=7]#[['phenologystageid','future_pheno']]

In [ ]:
for key in mejores_modelos.keys():
    print(key)
    datos_final['prediccion_'+str(key)]=mejores_modelos[key]['modelo'].predict(datos_final[mejores_modelos[key]['medidas'][:-1]], verbose=1)
#     for i in range(len(datos_final)):
        
        
datos_final    

In [ ]:
datos_final.columns[:20]

In [ ]:
horizon=7
preds_feno=datos_final.phenologystageid.values
for i in range(len(datos_final)):
    for key in mejores_modelos.keys():
        if key>preds_feno[i]:
            if datos_final['prediccion_'+str(key)].iloc[i]<horizon:
                preds_feno[i]=key
                
datos_final['preds_feno']=preds_feno

In [ ]:
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, roc_curve, confusion_matrix, classification_report, f1_score,recall_score

confusion_matrix(datos_final['future_pheno'],datos_final['preds_feno'])

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cf_matrix=confusion_matrix(datos_final['future_pheno'],datos_final['preds_feno'])
sns.heatmap(cf_matrix, annot=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))  
corres=datos_total2[['dia', 'tmed_min', 'tmed_max', 'tmed_mean', 'rad_min', 'rad_max', 'rad_mean','gdd_4.5_t0_Tbase_sum_Cumm',
 'gdd_4.5_t0_TbaseMax_sum_Cumm','gdd_4.5_1_Tbase_sum_Cumm','gdd_4.5_1_TbaseMax_sum_Cumm','gdd_4.5_2_Tbase_sum_Cumm',
 'gdd_4.5_2_TbaseMax_sum_Cumm','gdd_10.0_t0_Tbase_sum_Cumm','gdd_10.0_t0_TbaseMax_sum_Cumm','gdd_10.0_1_Tbase_sum_Cumm',
 'gdd_10.0_1_TbaseMax_sum_Cumm','gdd_10.0_2_Tbase_sum_Cumm','gdd_10.0_2_TbaseMax_sum_Cumm',
 'chillingDD_7.0_t0_Tbase_sum_Cumm','chillingDD_7.0_t0_Tbasemin_sum_Cumm','chillingDD_7.0_t0_Utah_sum_Cumm',
 'chillingDD_7.0_1_Tbase_sum_Cumm','chillingDD_7.0_1_Tbasemin_sum_Cumm','chillingDD_7.0_1_Utah_sum_Cumm',
 'chillingDD_7.0_2_Tbase_sum_Cumm','chillingDD_7.0_2_Tbasemin_sum_Cumm','chillingDD_7.0_2_Utah_sum_Cumm' , 'rad__t0__Cumm',
 'rad__1__Cumm', 'rad__2__Cumm', 'precip__t0__Cumm','precip__1__Cumm', 'precip__2__Cumm', 'winkler_4.5_t0_Tbase_Cumm',
 'winkler_4.5_t0_TbaseMax_Cumm','winkler_4.5_1_Tbase_Cumm','winkler_4.5_1_TbaseMax_Cumm','winkler_4.5_2_Tbase_Cumm',
 'winkler_4.5_2_TbaseMax_Cumm','winkler_10.0_t0_Tbase_Cumm','winkler_10.0_t0_TbaseMax_Cumm','winkler_10.0_1_Tbase_Cumm',
 'winkler_10.0_1_TbaseMax_Cumm','winkler_10.0_2_Tbase_Cumm','winkler_10.0_2_TbaseMax_Cumm', 'wind_N','wind_NE','wind_E',
 'wind_SE','wind_S','wind_SW','wind_W','wind_NW', 'longitude', 'latitude', 'altitud', 'phenologystageid']].corr().fillna(0)
sns.heatmap(corres[corres.columns[-2:]], annot=False, vmin=-1, vmax=1, square=True, ax=ax, cmap='seismic', linewidths=.5)